In [1]:
import dask
import logging
from datetime import datetime
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=10)

from grizly.scheduling.registry import Trigger, Job

## Job

In [6]:
@dask.delayed
def print_time():
    print(f"Hello, it's {datetime.now().__str__()}")
    
time_log = print_time()

@dask.delayed
def add(x, y):
    return x + y

_sum = add(1, 2)

In [7]:
job1 = Job("test_print")
job1.register(tasks=[time_log], 
              crons="* * * * *", 
              if_exists="replace")

job2 = Job("test_sum")
job2.register(tasks=[_sum], 
              upstream="test_print", 
              if_exists="replace")

2020-08-20 12:47:14,637 - grizly.scheduling.registry - DEBUG - Job(name='test_print') with cron '* * * * *' has been added to rq sheduler with id dd86622e-359a-47de-9e0e-5eacae7e4403
2020-08-20 12:47:14,642 - grizly.scheduling.registry - INFO - Job(name='test_print') has been added to the scheduler
2020-08-20 12:47:14,652 - grizly.scheduling.registry - INFO - Job(name='test_print') successfully registered
2020-08-20 12:47:14,677 - grizly.scheduling.registry - INFO - Adding ['test_sum'] to Job(name='test_print').downstream...
2020-08-20 12:47:14,694 - grizly.scheduling.registry - INFO - Job(name='test_sum') successfully registered


Job(name='test_sum')

In [8]:
job1.info()

name: test_print
owner: None
created_at: 2020-08-20 12:47:14.620190+00:00
crons: ['* * * * *']
downstream: [Job(name='test_sum')]
upstream: []
triggers: []


In [9]:
job1.runs

[]

In [10]:
job2.info()

name: test_sum
owner: None
created_at: 2020-08-20 12:47:14.660294+00:00
crons: []
downstream: []
upstream: [Job(name='test_print')]
triggers: []


In [11]:
job2.runs

[]

In [12]:
job1.submit(to_dask=False)

2020-08-20 12:47:15,223 - grizly.scheduling.registry - INFO - Submitting Job(name='test_print')...
2020-08-20 12:47:15,294 - grizly.scheduling.registry - INFO - Job(name='test_print') finished with status success
2020-08-20 12:47:15,297 - grizly.scheduling.registry - INFO - Enqueueing Job(name='test_print').downstream...
2020-08-20 12:47:15,308 - grizly.scheduling.registry - INFO - Job(name='test_sum') has been enqueued


Hello, it's 2020-08-20 12:47:15.287085


[None]

In [13]:
job1.last_run.info()

id: 1
name: None
created_at: 2020-08-20 12:47:15.229762+00:00
finished_at: 2020-08-20 12:47:15.313857+00:00
duration: 0
status: success
error: None


In [14]:
job1.unregister(remove_job_runs=True)

2020-08-20 12:47:15,577 - grizly.scheduling.registry - DEBUG - Rq job dd86622e-359a-47de-9e0e-5eacae7e4403 removed from the scheduler
2020-08-20 12:47:15,581 - grizly.scheduling.registry - INFO - Job(name='test_print') has been removed from the scheduler
2020-08-20 12:47:15,661 - grizly.scheduling.registry - INFO - Removing ['test_print'] from Job(name='test_sum').upstream...
2020-08-20 12:47:15,694 - grizly.scheduling.registry - INFO - Removing ['test_sum'] from Job(name='test_print').downstream...
2020-08-20 12:47:15,729 - grizly.scheduling.registry - INFO - Job(name='test_print') successfully removed from registry


In [15]:
job2.unregister()

2020-08-20 12:47:15,760 - grizly.scheduling.registry - INFO - Job(name='test_sum') successfully removed from registry
